# Assigning Reviewers to Candidates
[J. Nathan Matias](https://natematias.com), January 8, 2021

## Documentation
* Explanation for [how to set up the min cost flow algorithm for committee allocation](http://ozark.hendrix.edu/~yorgey/382/static/flow-network-application.pdf).

* ortools library documentation: [Assignment as a Minimum Cost Flow Problem](https://developers.google.com/optimization/flow/assignment_min_cost_flow)

## Illustration of the Min Cost Flow Diagram Used Here
<img src="flow_graph_illustration.jpg" alt="Drawing" style="width: 500px;"/>

In [8]:
import csv, os, sys, math, datetime
from collections import Counter, defaultdict
from ortools.graph.python import min_cost_flow
import pandas as pd
import random
random.seed(1729711011) #https://www.brooklynintegers.com/int/1729711011/

# in this graph the reviewers and the applications are nodes
# the review task is the edge 
# each review task or edge is given a cost 
# the cost of the edge is the preference for a reviewer (node) completing the review of that application (node)
# lower cost is a higher preference
# for the love of god run the cells in order
# I always forget this and debug individual cells

# Load Data
**anonymized-applicants.csv**: the applicant dataset needs the following columns:
* First Name
* Last Name
* Columns for prioritized reviewers (with valid reviewer ID):
  * 1 top
  * 2
  * 3
  * 4 lowest
  
**anonymized-reviewers.csv** needs the following columns:
* id (associated with the columns for prioritized reviewers)
* Full Review Quota (TRUE) or (FALSE)

In [9]:
applicant_file = "data/anonymizedApplicants_v2_test.csv"
# entering applicants into python list
applicants = []

with open(applicant_file) as f:
    for row in csv.DictReader(f):
        applicants.append(row)

# reads each row as a dictionary
# appends each row to an applicant list making a list of dictionaries

# randomly shuffle applicants
# in case there are systematic
# patterns in application order
# that would otherwise contribute
# to bias
random.shuffle(applicants)
#applicants stay in list

FileNotFoundError: [Errno 2] No such file or directory: 'data/anonymizedApplicants_v2_test.csv'

In [7]:
reviewer_file="data/anonymizedReviewers_v1_test.csv"
# entering applicants into python list
reviewers = []
with open(reviewer_file) as f:
    for row in csv.DictReader(f):
        reviewers.append(row)

# reads each row as a dictionary
# appends each row to an applicant list making a list of dictionaries
        
# randomly shuffle reviewers
# in case there are systematic
# patterns in reviewer order
# that would otherwise contribute
# to bias
random.shuffle(reviewers)
# applicants stay in list

reviewers_full = [x for x in reviewers if x['Full Review Quota']=="TRUE"]
reviewers_occasional = [x for x in reviewers if x['Full Review Quota']!="TRUE"]
print(reviewers)

[{'': '', 'Full Review Quota': 'TRUE', 'id': 'Nimbloom'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Ferrocorn'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Zazzlewing'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Quivelox'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Furrystomp'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Greezle'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Stardusta'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Sprocketfrost'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Twizzlefin'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'FluffisquillWhispertail'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Indigofin'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'TindraSparkplume'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Twirlbreeze'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Glimmersnort'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Thornwhisper'}, {'': '', 'Full Review Quota': 'TRUE', 'id': 'Twilighthorn'}, {'': '', 'Full Review Quota': 'TRUE'

# Validate Applicants File
Make sure that all of the recommended reviewers are in the reviewers dataset, and if not, list out applicants where this discrepancy exists.
As the allocator, you can then go into the spreadsheet and make any corrections as needed.

In [6]:
# Data entry check
reviewer_netids = [x['id'].lower() for x in reviewers]

validity_record = []

for applicant in applicants:
    applicant_fine = True
    for i in range(1,5):
        if applicant[str(i)] not in reviewer_netids:
            applicant_fine = False
    if(applicant_fine == False):
        print(applicant)
    validity_record.append(applicant_fine)
    
print("\n\n---------------------------\n\n")
# len is number of items or characters
print("{0} records have invalid reviewer suggestions".format(len([x for x in validity_record if x!=True])))

{'': '15', 'Last Name': 'heart', 'First Name': 'penetrate', '1': 'Snorblequill', '2': 'Twilighthorn', '3': 'Snufflewhisk', '4': 'Zazzlewing'}
{'': '24', 'Last Name': 'structure', 'First Name': 'public', '1': 'Thornwhisper', '2': 'Greezle', '3': 'Twilighthorn', '4': 'Quorblet'}
{'': '37', 'Last Name': 'tower', 'First Name': 'glow', '1': 'Stardusta', '2': 'Glimmerfawn', '3': 'Sprocketfrost', '4': 'Twirlbreeze'}
{'': '', 'Last Name': '', 'First Name': '', '1': '', '2': '', '3': '', '4': ''}
{'': '28', 'Last Name': 'appear', 'First Name': 'cinema', '1': 'Stardusta', '2': 'Sablette', '3': 'Nimbleshine', '4': 'Lunaswoop'}
{'': '22', 'Last Name': 'picture', 'First Name': 'an', '1': 'Greezle', '2': 'Twilighthorn', '3': 'Quorblet', '4': 'Twizzlefin'}
{'': '19', 'Last Name': 'sheet', 'First Name': 'leg', '1': 'Snorblequill', '2': 'Quivelox', '3': 'Zazzlewing', '4': 'Sprocketfrost'}
{'': '36', 'Last Name': 'pot', 'First Name': 'different', '1': 'Zazzlewing', '2': 'Snufflewhisk', '3': 'Twilighthor

## Basic Statistics

In [16]:
print("{0} total applicants".format(len(applicants)))
print("{0} available faculty for a full round of reviews".format(len(reviewers_full)))
print("{0} reviewers who can take a few".format(len(reviewers_occasional)))
print("Roughly {0} reviews per full reviewer faculty".format(math.floor(len(applicants) * 2 / len(reviewers_full))))

56 total applicants
27 available faculty for a full round of reviews
0 reviewers who can take a few
Roughly 4 reviews per full reviewer faculty


# Set up Graph

### Utility Methods

In [17]:
## Print vertex details for debugging
## C sharp module for python
## Creating nodes and edges for allocation of costs
def print_vertex(i):
    print("{0} -> {1} (capacity {2}, cost {3})".format(
        start_nodes[i], end_nodes[i], 
        capacities[i], costs[i]
    ))

## Scoring function that assigns priority cost for pairing applicant with reviewer
## Lower numbers are higher priority
## Important to note that 'applicant' and 'application' are used interchangeably

def reviewer_candidate_priority(applicant, reviewer):
    # default costs
    max_cost_full = 6
    max_cost_partial = 8
    if(reviewer['Full Review Quota']=="TRUE"):
        cost = max_cost_full
        # if reviewer is flagged as having a full quota then start at 6 otherwise start at 8
    else:
        cost = max_cost_partial
    reviewer_netid = reviewer['id'].lower().strip()
    # check if reviewer appears in preferences for that application
    for i in list(range(1,5)):
        if(reviewer_netid in applicant[str(i)].lower().strip()):
            cost = i
    ##TODO: Calculate topic overlaps to improve precision of matches
    return cost
# in this instance cost is the preference for pairing reviewer with application
# lower cost is a higher preference
# we favour the lower cost because economics

### Allocation Algorithm Settings

In [20]:
# set up enough capacity to handle all reviews across all full reviewers
# we can use the floor, since there are partial reviewers
# this calculation is fluid it works on the number of applications and is based on the number of reviewers per application
full_reviewer_assignment_count = math.floor(len(applicants) * 2 / len(reviewers_full))

# no more than five reviews per partial reviewer
partial_reviewer_assignment_count = 5 

total_tasks = len(applicants) * 2

reviews_per_applicant = 2


### Set up Cost Flow Graph

In [21]:
## All reviewers and applications need to be given node IDs
## on the same linear scale from 0..n
counter = 0
for idx, reviewer in enumerate(reviewers):
    reviewer['node_index'] = counter
    counter += 1
    # starts at 1 and adds an interger for each reviewer until n

counter = 0    
for applicant in applicants:
    applicant['node_index'] = counter
    counter += 1
    # starts at 1 and adds an interger for each application until n
# loops until all reviewers and applications are complete
## START_NODES AND END_NODES:
## tasks flow from the source (index 0) to the sink (last index)
source_index = source = 0
sink = sink_index = counter


## enter reviewers and applications into dataframe
start_nodes = []
end_nodes   = []

## CAPACITIES: how many tasks can flow across node
## Each reviewer can take on full_reviewer_assignment_count reviewers
##      from the Source.
## Each applicant can take on only one review from one faculty
## The sink can take a variable number of reviews from each applicant
## Number of reviews depends on £5k or £70k grant
## SOURCE IS REVIEWERS
capacities = []

## COSTS: proxy for priority, where higher priority = lower cost
## on a scale from 0 to N
## uses information from Utility Methods section
# assigns cost to each task where a task is an edge
costs = []

## First, add nodes from the source (index 0) to the reviewers
## start at index 0 because python
## making reviewers nodes in the graph
for reviewer in reviewers:
    start_nodes.append(source_index)
    end_nodes.append(reviewer['node_index'])
    # Taking into account priorities from Utility Methods but we need to assume all reviewers will be full
    if(reviewer['Full Review Quota']=="TRUE"):
        capacities.append(full_reviewer_assignment_count)
    else:
        capacities.append(partial_reviewer_assignment_count)
        
    # no cost to allocate from the source
    costs.append(0)
        
    
## now add vertices from each reviewer to each applicant:
## applicants become nodes in the graph
for reviewer in reviewers:
    for applicant in applicants:
        start_nodes.append(reviewer['node_index'])
        end_nodes.append(applicant['node_index'])
        # only one review from each reviewer
        capacities.append(1)
        
        ## cost for a given reviewer applicant pair
        costs.append(reviewer_candidate_priority(applicant, reviewer))
        
## now add vertices from each applicant to the sink
## SINK IS APPLICATIONS
for applicant in applicants:
    start_nodes.append(applicant['node_index'])
    end_nodes.append(sink_index)
    # N applications per candidate
    # set in section allocation algorithm settings line 11
    capacities.append(reviews_per_applicant)
    
    # no cost to reach the sink
    costs.append(0)
    
    
## SET SUPPLIES: This is a vector with a single count
## for the number of supplies available at each node

## set the number of supplies at the source
supplies    = [total_tasks]

## set reviewers and applicant supplies to zero
for reviewer in reviewers:
    supplies.append(0)
for applicant in applicants:
    supplies.append(0)

## set the sink supply to zero
# will run through from zero because python
supplies.append(total_tasks*-1)

NameError: name 'counter' is not defined

### Confirm validity of graph

In [14]:
# Check that the start node, end node, costs, and capacity have the same length
# Having the same length is important because they represent parallel arrays
# If they do not have the same length there is a mis-match in allocation
# will return true/false
# if false then calc will fail
print("{0}: start node, end node, cost, and capacity all have equal length".format(len(start_nodes) == len(end_nodes) == len(costs) == len(capacities)))

True: start node, end node, cost, and capacity all have equal length


In [14]:
## confirm that the source is set up properly
## source is total number of tasks ie reviews
# each link from the source to a reviewer
# initial number of reviews per reviewer with no cost allocated to them
# should have a cost of 0
# should show capacity of each reviewer without any costs
for i in range(0,len(reviewers)):
    print_vertex(i)

0 -> 1 (capacity 12, cost 0)
0 -> 2 (capacity 12, cost 0)
0 -> 3 (capacity 5, cost 0)
0 -> 4 (capacity 12, cost 0)
0 -> 5 (capacity 12, cost 0)
0 -> 6 (capacity 12, cost 0)
0 -> 7 (capacity 12, cost 0)
0 -> 8 (capacity 12, cost 0)
0 -> 9 (capacity 12, cost 0)
0 -> 10 (capacity 12, cost 0)
0 -> 11 (capacity 5, cost 0)
0 -> 12 (capacity 12, cost 0)
0 -> 13 (capacity 5, cost 0)
0 -> 14 (capacity 12, cost 0)
0 -> 15 (capacity 5, cost 0)
0 -> 16 (capacity 12, cost 0)
0 -> 17 (capacity 12, cost 0)
0 -> 18 (capacity 12, cost 0)
0 -> 19 (capacity 12, cost 0)
0 -> 20 (capacity 12, cost 0)
0 -> 21 (capacity 5, cost 0)
0 -> 22 (capacity 5, cost 0)


In [15]:
## confirm some of the reviewer to applicant links (reviewer 3)
#  the capacity for each should be 1 and the cost should vary
# displays that costs ie preferences have been calculated correctly
for i in range(len(reviewers) + len(applicants)*3, len(reviewers) + len(applicants*3) + 20):
    print_vertex(i)

4 -> 23 (capacity 1, cost 6)
4 -> 24 (capacity 1, cost 6)
4 -> 25 (capacity 1, cost 6)
4 -> 26 (capacity 1, cost 6)
4 -> 27 (capacity 1, cost 6)
4 -> 28 (capacity 1, cost 3)
4 -> 29 (capacity 1, cost 6)
4 -> 30 (capacity 1, cost 6)
4 -> 31 (capacity 1, cost 6)
4 -> 32 (capacity 1, cost 6)
4 -> 33 (capacity 1, cost 1)
4 -> 34 (capacity 1, cost 6)
4 -> 35 (capacity 1, cost 6)
4 -> 36 (capacity 1, cost 3)
4 -> 37 (capacity 1, cost 6)
4 -> 38 (capacity 1, cost 6)
4 -> 39 (capacity 1, cost 6)
4 -> 40 (capacity 1, cost 6)
4 -> 41 (capacity 1, cost 6)
4 -> 42 (capacity 1, cost 6)


In [15]:
## confirm that the sink is set up properly
# each applicant should have a capacity of 2 and cost of 0
# this is showing the number of reviews allocated to each application
for i in range(len(start_nodes) - len(applicants), len(start_nodes)-1):
    print_vertex(i)

23 -> 123 (capacity 2, cost 0)
24 -> 123 (capacity 2, cost 0)
25 -> 123 (capacity 2, cost 0)
26 -> 123 (capacity 2, cost 0)
27 -> 123 (capacity 2, cost 0)
28 -> 123 (capacity 2, cost 0)
29 -> 123 (capacity 2, cost 0)
30 -> 123 (capacity 2, cost 0)
31 -> 123 (capacity 2, cost 0)
32 -> 123 (capacity 2, cost 0)
33 -> 123 (capacity 2, cost 0)
34 -> 123 (capacity 2, cost 0)
35 -> 123 (capacity 2, cost 0)
36 -> 123 (capacity 2, cost 0)
37 -> 123 (capacity 2, cost 0)
38 -> 123 (capacity 2, cost 0)
39 -> 123 (capacity 2, cost 0)
40 -> 123 (capacity 2, cost 0)
41 -> 123 (capacity 2, cost 0)
42 -> 123 (capacity 2, cost 0)
43 -> 123 (capacity 2, cost 0)
44 -> 123 (capacity 2, cost 0)
45 -> 123 (capacity 2, cost 0)
46 -> 123 (capacity 2, cost 0)
47 -> 123 (capacity 2, cost 0)
48 -> 123 (capacity 2, cost 0)
49 -> 123 (capacity 2, cost 0)
50 -> 123 (capacity 2, cost 0)
51 -> 123 (capacity 2, cost 0)
52 -> 123 (capacity 2, cost 0)
53 -> 123 (capacity 2, cost 0)
54 -> 123 (capacity 2, cost 0)
55 -> 12

In [16]:
## confirm that the sum of the supplies is zero
## total supply must equal total demand giving zero in a sum
print("{0}: sum of the supplies is zero".format(sum(supplies)==0))
print("{0}: correct number of supplies".format(len(set(start_nodes)) +1 == len(set(end_nodes)) + 1 == len(supplies)))
print(supplies)

True: sum of the supplies is zero
True: correct number of supplies
[200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -200]


## Set up min cost flow object
In a valid graph:
* The capacity can be greater than the actual flows
* The supply and the sink need to be equal
* All of the supply needs to flow to the sink

In [20]:
# this cell checks that each reviewer is doing the correct number of reviews 
# assigns reviews according to cost
import numpy as np
from ortools.graph.python import min_cost_flow

# Instantiate the solver (object, not module)
smcf = min_cost_flow.SimpleMinCostFlow()

# Add each arc (instance method, snake_case)
# this has been calculated above in cell 16
for i in range(len(start_nodes)):
    smcf.add_arc_with_capacity_and_unit_cost(
        int(start_nodes[i]),
        int(end_nodes[i]),
        int(capacities[i]),
        int(costs[i])
    )

# Add node supplies (instance method, snake_case)
for i in range(len(supplies)):
    smcf.set_node_supply(int(i), int(supplies[i]))

# Solve
status = smcf.solve()

if status == min_cost_flow.SimpleMinCostFlow.OPTIMAL:
    print("Optimal cost:", smcf.optimal_cost())
    print("Max flow:", smcf.maximum_flow())
    # Inspect per‑arc fields
    for a in range(smcf.num_arcs()):
        print(f"arc {a}: {smcf.tail(a)} -> {smcf.head(a)}, "
              f"cap={smcf.capacity(a)}, flow={smcf.flow(a)}, cost={smcf.unit_cost(a)}")
else:
    print ('There was an issue with the calculation')



Optimal cost: 341
Max flow: 200
arc 0: 0 -> 1, cap=12, flow=12, cost=0
arc 1: 0 -> 2, cap=12, flow=12, cost=0
arc 2: 0 -> 3, cap=5, flow=1, cost=0
arc 3: 0 -> 4, cap=12, flow=11, cost=0
arc 4: 0 -> 5, cap=12, flow=12, cost=0
arc 5: 0 -> 6, cap=12, flow=10, cost=0
arc 6: 0 -> 7, cap=12, flow=12, cost=0
arc 7: 0 -> 8, cap=12, flow=12, cost=0
arc 8: 0 -> 9, cap=12, flow=12, cost=0
arc 9: 0 -> 10, cap=12, flow=12, cost=0
arc 10: 0 -> 11, cap=5, flow=4, cost=0
arc 11: 0 -> 12, cap=12, flow=12, cost=0
arc 12: 0 -> 13, cap=5, flow=0, cost=0
arc 13: 0 -> 14, cap=12, flow=12, cost=0
arc 14: 0 -> 15, cap=5, flow=5, cost=0
arc 15: 0 -> 16, cap=12, flow=12, cost=0
arc 16: 0 -> 17, cap=12, flow=12, cost=0
arc 17: 0 -> 18, cap=12, flow=12, cost=0
arc 18: 0 -> 19, cap=12, flow=12, cost=0
arc 19: 0 -> 20, cap=12, flow=12, cost=0
arc 20: 0 -> 21, cap=5, flow=0, cost=0
arc 21: 0 -> 22, cap=5, flow=1, cost=0
arc 22: 1 -> 23, cap=1, flow=0, cost=6
arc 23: 1 -> 24, cap=1, flow=0, cost=6
arc 24: 1 -> 25, ca

In [21]:
smcf.num_arcs()

2322

## (optional) Output Dotfile of Graph
This dotfile can be loaded into Gephi or output to GraphVis in order to debug and confirm that the solution is acceptable.

In [22]:
import numpy as np
from ortools.graph.python import min_cost_flow

def applicant_name(applicant):
    return applicant['First Name'].replace(" ", "_").replace("-", "_") + "_" + applicant['Last Name'].replace(" ", "_").replace("-", "_")

status = smcf.solve()
if status == min_cost_flow.SimpleMinCostFlow.OPTIMAL:
    print('Total cost = ', smcf.optimal_cost())

    ## output to dotfile
    with open("data/{0}-allocation-graph.dot".format(int(datetime.datetime.now().timestamp())), "w") as f:

        print("digraph g{", file=f)
        for reviewer in reviewers:
            print("{} [type=reviewer];".format(reviewer['id'].replace("-","_")), file=f)
        for applicant in applicants:
            print("{} [type=applicant];".format(applicant_name(applicant).replace("-","_")), file=f)


        for arc in range(smcf.num_arcs()):

          # Can ignore arcs leading out of source or into sink.
         if smcf.tail(arc)!=source and smcf.head(arc)!=sink:

            # Arcs in the solution have a flow value of 1. Their start and end nodes
            # give an assignment of worker to task.

            if smcf.flow(arc) > 0:
              applicant = applicants[smcf.head(arc) - len(reviewers) - 1]
              print('%s -> %s [weight = %d];' % (
                    reviewers[smcf.tail(arc)-1]['id'].replace("-","_"),
                    (applicant_name(applicant).replace("-","_")),
                    smcf.unit_costost(arc)), file=f)
        print("}", file=f)
else:
    print('There was an issue with the min cost flow input.')

Total cost =  341


AttributeError: 'ortools.graph.python.min_cost_flow.SimpleMinCostFlow' object has no attribute 'unit_costost'

# Output Applicant Spreadsheet with Assignment Columns
This code takes the applicant dataset and adds two columns to it:
* Reviewer 1
* Reviewer 2

These are the final reviewers. **Reviewers are not listed in any particular order**.

### Add assignment columns to list of dicts

In [23]:

for applicant in applicants:
    if('reviewer 1' in applicant.keys()):
        del applicant['reviewer 1']
    if('reviewer 2'in applicant.keys()):
        del applicant['reviewer 2']

In [29]:
status = smcf.solve()
if status == min_cost_flow.SimpleMinCostFlow.OPTIMAL:
    print('Total cost = ', smcf.optimal_cost())
    
    for arc in range(smcf.num_arcs()):
     # ignore arcs leading out of source or into sink.
     if smcf.tail(arc)!=source and smcf.head(arc)!=sink:
        # Arcs in the solution have a flow value of 1. Their start and end nodes
        # give an assignment of worker to task.
        # loops to assign according to priority whilst ensuring number of reviews isn't exceeded
        if smcf.flow(arc) > 0:
            applicant = applicants[smcf.head(arc) - len(reviewers) - 1]
            if('reviewer 1' not in applicant.keys()):
                applicant['reviewer 1'] = reviewers[smcf.tail(arc)-1]['id']
                applicant['priority 1'] = smcf.unit_cost(arc) 
            else:
                applicant['reviewer 2'] = reviewers[smcf.tail(arc)-1]['id']
                applicant['priority 2'] = smcf.unit_cost(arc) 

Total cost =  341


### Check Balance of reviews per faculty

In [30]:
# checks reviewer data present for the applications
applicant.keys()

dict_keys(['', 'Last Name', 'First Name', '1', '2', '3', '4', 'node_index', 'reviewer 1', 'reviewer 2', 'priority 2'])

In [31]:
# checks number of tasks per reviewer
reviewer_reviews = defaultdict(list)
for applicant in applicants:
    reviewer_reviews[applicant['reviewer 1']].append(applicant['Last Name'] + ", " + applicant['First Name'])
    reviewer_reviews[applicant['reviewer 2']].append(applicant['Last Name'] + ", " + applicant['First Name'])

for reviewer, review_names in reviewer_reviews.items():
    print("{0}: {1}".format(reviewer, len(review_names)))

7d50df7d-8b0d-4b43-a90e-375298ecddb3: 12
8008686c-7de9-42d8-b27a-2abbebd44e33: 12
eaba192c-e945-4c8a-a800-47907aa33c50: 12
e5203f9d-8874-4f97-882a-fceb05863ea5: 12
ee2b28ae-0630-489a-9287-d6e677404815: 12
2da9f25d-e3cf-4633-8e15-233157071d75: 12
2fec5177-e9a6-45cc-9dfd-bff6fe5f4bc0: 11
b44a1a08-334f-4223-93e1-1fde6f090f75: 5
42afa2ed-5735-42df-b6a4-e1b6b6e6d151: 4
4282ce75-1089-4aac-9bfe-55e65a6312d4: 12
a13619cd-e1eb-4b49-9a19-e0e423577037: 12
894ed6f3-b99c-44fc-beb5-1e4858e20224: 12
139b90b3-ca25-4ccb-97c9-0dbf3fa1cc3a: 12
47d768f6-3d10-4a8a-90ed-31388f9ae08a: 1
3516c055-4219-4558-bf60-71f2379c8e29: 12
5338d673-a089-4ad4-9d6c-a8184fd16a4d: 10
bee21ae1-487a-454c-a17d-1d097d9045ab: 12
8e826cfa-608b-4409-9203-4ae1b7c5550a: 12
34a6e61c-0ee3-4803-8728-e6ac8f7582d3: 12
45b7d43f-50d4-49e3-91ca-3a4e2d8bc810: 1


# Write to CSV

In [65]:
pd.DataFrame(applicants).to_csv("data/{0}-applicant-reviewer-allocations.csv".format(int(datetime.datetime.now().timestamp())))